Batch normalized, ridge learning rate for outer layer, he initialised.

initially a fully connected MLP is trained for 20 epochs then we introduce pruning based on the weights magnitude(weak ones deleted). The new links are made in the non adjacent layers. and weights are 'he' initialised
Asynchnorous learning: layers involving outer layer ridge regression based learning function, others regular sgd. Accuracy increased till 65.18%

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
import os

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.linear_model import Ridge

# Enable GPU memory growth
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"✅ Running on GPU: {physical_devices[0].name}")
else:
    print("⚠️ No GPU found. Running on CPU.")

class MLP_CIFAR10:
    def __init__(self):
        self.batch_size = 100
        self.maxepoches = 100
        self.initial_epochs = 20
        self.learning_rate = 0.01
        self.num_classes = 10
        self.momentum = 0.9
        self.prune_percent = 0.1  # 10% pruning

        self.create_model()
        self.train()

    def create_model(self):
        self.model = Sequential()
        self.model.add(Flatten(input_shape=(32, 32, 3)))
        self.model.add(Dense(4000, kernel_initializer='he_uniform'))
        self.model.add(BatchNormalization())                     # Layer 1
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(1000, kernel_initializer='he_uniform'))
        self.model.add(BatchNormalization())                 # Layer 2
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(4000, kernel_initializer='he_uniform'))
        self.model.add(BatchNormalization())               # Layer 3
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(self.num_classes, activation='softmax', kernel_initializer='he_uniform', name="output_layer"))

    def train(self):
        x_train, y_train, x_test, y_test = self.read_data()

        datagen = ImageDataGenerator(
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True
        )
        datagen.fit(x_train)

        # Freeze output layer
        self.model.get_layer("output_layer").trainable = False

        # Print model summary and active parameters
        self.model.summary()
        print(f"\n[INFO] Initial active parameters: {self.count_total_nonzero_weights()}")

        sgd = SGD(learning_rate=self.learning_rate, momentum=self.momentum)
        self.model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

        print("\n[PHASE 1] Initial Training")
        self.model.fit(
            datagen.flow(x_train, y_train, batch_size=self.batch_size),
            epochs=self.initial_epochs,
            validation_data=(x_test, y_test),
            steps_per_epoch=len(x_train) // self.batch_size,
            verbose=2
        )

        print("\n[PHASE 2] Pruning and Rewiring")
        total_before = self.count_total_nonzero_weights()
        self.prune_weights(self.prune_percent)
        total_after_prune = self.count_total_nonzero_weights()
        rewiring_needed = total_before - total_after_prune
        self.rewire_model_balanced(rewiring_needed)
        total_after_rewire = self.count_total_nonzero_weights()

        print(f"[INFO] Non-zero weights: before={total_before}, after prune={total_after_prune}, after rewire={total_after_rewire}")

        print("\n[PHASE 3] Final Training")
        history = self.model.fit(
            datagen.flow(x_train, y_train, batch_size=self.batch_size),
            epochs=self.maxepoches,
            initial_epoch=self.initial_epochs,
            validation_data=(x_test, y_test),
            steps_per_epoch=len(x_train) // self.batch_size,
            verbose=2
        )

        # Train output layer using Ridge Regression
        self.train_output_layer_with_ridge(x_train, y_train)

        self.accuracies_per_epoch = history.history['val_accuracy']

    def train_output_layer_with_ridge(self, x_train, y_train, alpha=1.0):
        feature_model = Model(inputs=self.model.input, outputs=self.model.get_layer(index=-2).output)
        features = feature_model.predict(x_train, batch_size=self.batch_size)

        ridge = Ridge(alpha=alpha, fit_intercept=True)
        ridge.fit(features, y_train)

        weights = ridge.coef_.T
        biases = ridge.intercept_
        self.model.get_layer("output_layer").set_weights([weights, biases])
        print("[INFO] ✅ Output layer learned via Ridge Regression.")

    def read_data(self):
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        y_train = to_categorical(y_train, self.num_classes)
        y_test = to_categorical(y_test, self.num_classes)
        x_train = x_train.astype('float32') / 255.0
        x_test = x_test.astype('float32') / 255.0
        return x_train, y_train, x_test, y_test

    def count_total_nonzero_weights(self):
        total = 0
        for layer in self.model.layers:
            if isinstance(layer, Dense):
                W, _ = layer.get_weights()
                total += np.count_nonzero(W)
        return total

    def prune_weights(self, prune_percent=0.2):
        for layer in self.model.layers:
            if isinstance(layer, Dense):
                weights, biases = layer.get_weights()
                flat = np.abs(weights.flatten())
                threshold = np.percentile(flat, prune_percent * 100)
                weights[np.abs(weights) < threshold] = 0.0
                layer.set_weights([weights, biases])

    def he_uniform_init(self, fan_in):
        limit = np.sqrt(6.0 / fan_in)
        return np.random.uniform(-limit, limit)

    def rewire_model_balanced(self, rewiring_needed, seed=42):
        print(f"\n[INFO] Rewiring {rewiring_needed} connections with HE initialization...")

        np.random.seed(seed)
        dense_layers = [layer for layer in self.model.layers if isinstance(layer, Dense)]
        layer_weights = [layer.get_weights() for layer in dense_layers]

        valid_layer_pairs = [(i, j) for i in range(len(dense_layers))
                            for j in range(len(dense_layers)) if abs(i - j) > 1]

        per_pair = rewiring_needed // len(valid_layer_pairs)
        connections_added = 0

        for (src_idx, dst_idx) in valid_layer_pairs:
            W_src, _ = layer_weights[src_idx]
            W_dst, b_dst = layer_weights[dst_idx]

            src_neurons = np.where(np.any(W_src != 0, axis=0))[0]
            dst_neurons = np.where(np.any(W_dst == 0, axis=1))[0]

            if len(src_neurons) == 0 or len(dst_neurons) == 0:
                continue

            k = min(per_pair, len(src_neurons) * len(dst_neurons))
            new_dst_idxs = np.random.choice(dst_neurons, size=k, replace=True)
            new_src_idxs = np.random.choice(src_neurons, size=k, replace=True)

            for d, s in zip(new_dst_idxs, new_src_idxs):
                if W_dst[d, s % W_dst.shape[1]] == 0:
                    fan_in = W_src.shape[0]
                    W_dst[d, s % W_dst.shape[1]] = self.he_uniform_init(fan_in)
                    connections_added += 1

            layer_weights[dst_idx][0] = W_dst

        for i, layer in enumerate(dense_layers):
            layer.set_weights(layer_weights[i])

        print(f"[INFO] ✅ Rewiring complete. Connections added: {connections_added}")
        print(f"[INFO] Total active parameters after rewiring: {self.count_total_nonzero_weights()}\n")

def main():
    print("\n=== Sparse MLP with Ridge Output Layer for CIFAR-10 ===")
    model = MLP_CIFAR10()

    os.makedirs("results", exist_ok=True)
    np.savetxt("results/dense_mlp_balanced_pruned_rewired_acc.txt",
               np.asarray(model.accuracies_per_epoch))

    print("\n=== Training Complete ===")
    print(f"Final model has {model.count_total_nonzero_weights()} active parameters")
    print("Results saved to 'results/dense_mlp_balanced_pruned_rewired_acc.txt'")

if __name__ == '__main__':
    main()


⚠️ No GPU found. Running on CPU.

=== Sparse MLP with Ridge Output Layer for CIFAR-10 ===


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
